In [15]:
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.model_selection
import sklearn.neural_network

In [16]:
features = pd.read_csv('hb_train_feature.csv')
truths = pd.read_csv('hb_train_truth.csv')

In [4]:
# add features to data

In [17]:
# add unique NE feature
!python -m pip install nashpy
import nashpy
import warnings
warnings.filterwarnings('ignore')

UNEs, NE_cnts = [], []
for i in range(features.shape[0]):
    payoffs = np.array(features.iloc[i]) 
    R = payoffs[:9].reshape(3,3)
    C = payoffs[9:18].reshape(3,3)
    rps = nashpy.Game(R, C)
    eqs = list(rps.support_enumeration()) # could be unique or multiple (PNE MNE)
    UNE = list(np.concatenate(eqs[0])) if len(eqs)==1 else list(np.zeros(6))
    NE_cnt = len(eqs)    
    UNEs.append(UNE)
    NE_cnts.append(NE_cnt)

# append to features
names = ['UNE_r1', 'UNE_r2','UNE_r3','UNE_c1','UNE_c2','UNE_c3']

for i in range(6):
    features[names[i]] = [UNE[i] for UNE in UNEs]
features['NE_cnts'] = NE_cnts

In [18]:
# add max max feature
features['max_max'] = features.iloc[:,:9].idxmax(axis=1).apply(lambda x: int(x[1]))
features = pd.get_dummies(features, columns=['max_max'], drop_first=True)
features.head()

,r11,r12,r13,r21,r22,r23,r31,r32,r33,c11,...,c33,UNE_r1,UNE_r2,UNE_r3,UNE_c1,UNE_c2,UNE_c3,NE_cnts,max_max_2,max_max_3
0,80.0,50.0,80.0,20.0,90.0,60.0,30.0,10.0,10.0,70.0,...,50.0,1.0,0.0,0.0,1.0,0.0,0.0,1,1,0
1,20.0,51.0,50.0,10.0,70.0,30.0,60.0,5.0,45.0,20.0,...,45.0,0.0,0.0,0.0,0.0,0.0,0.0,5,1,0
2,70.0,100.0,48.0,10.0,29.0,40.0,40.0,60.0,100.0,70.0,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,0
3,20.0,60.0,70.0,30.0,90.0,20.0,40.0,40.0,50.0,40.0,...,60.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,0
4,13.0,80.0,60.0,74.0,20.0,70.0,60.0,100.0,10.0,13.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,1


In [19]:
# # add minimax feature
def grid_form(row):
    return np.array(row).reshape(3, 3)

def is_argmax(arr, i):
    return arr[i] == max(arr)
               
def is_min_max(row, i):
    grid = grid_form(row)
    mins = np.min(grid, axis=0)
    if is_argmax(mins, i - 1):
        return 1
    return 0
        
for i in range(1,4):
    features['min_max_{}'.format(i)] = 0
    for j in range(len(features)):
        features.loc[j,'min_max_{}'.format(i)] = is_min_max(features.iloc[j, :9], i)

features.head()

,r11,r12,r13,r21,r22,r23,r31,r32,r33,c11,...,UNE_r3,UNE_c1,UNE_c2,UNE_c3,NE_cnts,max_max_2,max_max_3,min_max_1,min_max_2,min_max_3
0,80.0,50.0,80.0,20.0,90.0,60.0,30.0,10.0,10.0,70.0,...,0.0,1.0,0.0,0.0,1,1,0,1,0,0
1,20.0,51.0,50.0,10.0,70.0,30.0,60.0,5.0,45.0,20.0,...,0.0,0.0,0.0,0.0,5,1,0,0,0,1
2,70.0,100.0,48.0,10.0,29.0,40.0,40.0,60.0,100.0,70.0,...,0.0,0.0,0.0,0.0,3,0,0,0,0,1
3,20.0,60.0,70.0,30.0,90.0,20.0,40.0,40.0,50.0,40.0,...,0.0,0.0,0.0,0.0,2,1,0,0,1,0
4,13.0,80.0,60.0,74.0,20.0,70.0,60.0,100.0,10.0,13.0,...,0.0,0.0,0.0,0.0,3,0,1,0,1,0


In [20]:
## add maximum payoff for both agents feature
def is_max_altruism(row, i):
    print(row)
    total_welfare = [row[i] + row[i + 9] for i in range(9)]
    if is_argmax(i):
        return 1
    return 0    

for i in range(1,4):
    features['altruism_{}'.format(i)] = 0
    for j in range(len(features)):
        features.loc[j,'altruism_{}'.format(i)] = is_min_max(features.iloc[j, :9], i)
        
features.head()

,r11,r12,r13,r21,r22,r23,r31,r32,r33,c11,...,UNE_c3,NE_cnts,max_max_2,max_max_3,min_max_1,min_max_2,min_max_3,altruism_1,altruism_2,altruism_3
0,80.0,50.0,80.0,20.0,90.0,60.0,30.0,10.0,10.0,70.0,...,0.0,1,1,0,1,0,0,1,0,0
1,20.0,51.0,50.0,10.0,70.0,30.0,60.0,5.0,45.0,20.0,...,0.0,5,1,0,0,0,1,0,0,1
2,70.0,100.0,48.0,10.0,29.0,40.0,40.0,60.0,100.0,70.0,...,0.0,3,0,0,0,0,1,0,0,1
3,20.0,60.0,70.0,30.0,90.0,20.0,40.0,40.0,50.0,40.0,...,0.0,2,1,0,0,1,0,0,1,0
4,13.0,80.0,60.0,74.0,20.0,70.0,60.0,100.0,10.0,13.0,...,0.0,3,0,1,0,1,0,0,1,0


In [31]:
# splits into training and test sets
X_training, X_test = sk.model_selection.train_test_split(features)
y_training, y_test = truths[['f1', 'f2', 'f3']].loc[X_training.index], truths[['f1', 'f2', 'f3']].loc[X_test.index]

# standardizes features 
scaler = sk.preprocessing.StandardScaler()
scaler.fit_transform(X_training)
scaler.transform(X_test)
scaler.transform(X_training);
#X_training.head()

In [28]:
# trains separate neural net to predict each specific player's actions
classifier = sk.neural_network.MLPRegressor(solver='lbfgs', random_state=1, max_iter=1000000000, verbose=True)
classifier.fit(X_training, y_training)
classifier.score(X_test, y_test)


-7.575382045673166

In [35]:
# function to test # of top actions predicted correctly
top_actions_test = truths['action'].loc[y_test.index].reset_index(drop=True)
predicted_frequency = pd.DataFrame(classifier.predict(X_test))
predicted_actions = predicted_frequency.idxmax(axis=1).apply(lambda x: x + 1)

# number of correct top-action predictions on test data
result = (top_actions_test==predicted_actions).value_counts()
print("A =", result[1]/(result[0]+result[1]))

A = 0.8412698412698413


In [40]:
# cross_validation
from sklearn.model_selection import cross_val_score
classifier = sk.neural_network.MLPRegressor(solver='lbfgs', random_state=1, max_iter=1000000000, verbose=True)
scores = cross_val_score(classifier, features, truths)

print('Cross-Validation scores: {}'.format(scores))
print('Average score: {}'.format(np.mean(scores)))

Cross-Validation scores: [-2.14716592 -2.75693201 -3.84906817 -3.90371751 -3.07026094]
Average score: -3.1454289105989583


In [72]:
# Apply the trained model to training data
predicted_frequency = pd.DataFrame(classifier.predict(features))

# calculate A score on training data
truth_actions = truths['action']
predicted_actions = predicted_frequency.idxmax(axis=1).apply(lambda x: x + 1)
As = (truth_actions==predicted_actions)
print("A =", np.mean(As))

# calculate Q score on training data
truth_freqency = truths[['f1', 'f2', 'f3']]
predicted_frequency.columns = ['f1', 'f2', 'f3']
Qs = ((truth_freqency.subtract(predicted_frequency, axis='column'))**2).sum(axis=1)
print("Q =", np.mean(Qs))

A = 0.852
Q = 0.4405986312907718
